In [2]:
import tensorflow as tf
import timeit

# GPU가 사용 가능한지 확인합니다.
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
# CPU 벤치마크 수행
def cpu_benchmark():
    with tf.device('/CPU:0'):
        random_matrix = tf.random.normal([1000, 1000])
        dot_operation = tf.matmul(random_matrix, tf.transpose(random_matrix))
        return dot_operation

cpu_time = timeit.timeit(lambda: cpu_benchmark(), number=10)
print("CPU Execution Time: {} sec".format(cpu_time))

# GPU 벤치마크 수행
def gpu_benchmark():
    with tf.device('/GPU:0'):
        random_matrix = tf.random.normal([1000, 1000])
        dot_operation = tf.matmul(random_matrix, tf.transpose(random_matrix))
        return dot_operation

gpu_time = timeit.timeit(lambda: gpu_benchmark(), number=10)
print("GPU Execution Time: {} sec".format(gpu_time))


Num GPUs Available:  0
CPU Execution Time: 0.11357283399996732 sec
GPU Execution Time: 0.09035437500000398 sec


In [1]:
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds

tf.enable_v2_behavior()

from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()


(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

batch_size = 128

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(batch_size)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)


ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(batch_size)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)


model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, kernel_size=(3, 3),
                 activation='relu'),
  tf.keras.layers.Conv2D(64, kernel_size=(3, 3),
                 activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
#   tf.keras.layers.Dropout(0.25),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
#   tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=['accuracy'],
)

model.fit(
    ds_train,
    epochs=12,
    validation_data=ds_test,
)


NotFoundError: dlopen(/Users/sj/opt/anaconda3/envs/magic/lib/python3.8/site-packages/tensorflow-plugins/libmetal_plugin.dylib, 0x0006): symbol not found in flat namespace '_TF_GetInputPropertiesList'

In [ ]:
!rm -r /Users/sj/opt/anaconda3/envs/magic/lib/python3.8/site-packages/tensorflow-plugins
!pip uninstall tensorflow-metal
!pip install tensorflow-metal
y